# 📈 04. Time Series Forecast with ARIMA

In this notebook, we apply an ARIMA model to the USD/JPY exchange rate to forecast future values.  
（このノートでは、USD/JPYの時系列データにARIMAモデルを適用して予測を行います）

In [ ]:
!pip install yfinance statsmodels --quiet

In [ ]:
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
from datetime import timedelta

In [ ]:
# USD/JPY の日次データ取得（2023年分）
df = yf.download("USDJPY=X", start="2023-01-01", end="2023-12-31")
df = df[['Close']].dropna()
df.head()

In [ ]:
# 時系列データのインデックスを確認
df.index = pd.to_datetime(df.index)
ts = df['Close']

In [ ]:
# ARIMA(p=5, d=1, q=0) を例に使用（最適化は次ステップ）
model = sm.tsa.ARIMA(ts, order=(5, 1, 0))
results = model.fit()

forecast_steps = 10
forecast = results.get_forecast(steps=forecast_steps)
predicted_mean = forecast.predicted_mean
conf_int = forecast.conf_int()

In [ ]:
# 日付インデックスの生成
last_date = ts.index[-1]
future_dates = [last_date + timedelta(days=i) for i in range(1, forecast_steps+1)]

# DataFrame化
forecast_df = pd.DataFrame({
    'Forecast': predicted_mean.values,
    'Lower CI': conf_int.iloc[:, 0],
    'Upper CI': conf_int.iloc[:, 1]
}, index=future_dates)

forecast_df.head()

In [ ]:
plt.figure(figsize=(14,6))
plt.plot(ts, label='Actual')
plt.plot(forecast_df['Forecast'], label='Forecast', color='orange')
plt.fill_between(forecast_df.index, forecast_df['Lower CI'], forecast_df['Upper CI'],
                 color='gray', alpha=0.3, label='Confidence Interval')
plt.title('USD/JPY Forecast using ARIMA')
plt.xlabel('Date')
plt.ylabel('Exchange Rate')
plt.legend()
plt.grid(True)
plt.show()

## ✅ Summary

- ARIMA(5,1,0) was applied to USD/JPY closing price data
- A 10-day forecast was generated with confidence intervals
- You can tune the model or try other ARIMA parameters

（このノートでは、ARIMAモデルで為替レートの10日先を予測し、グラフで可視化しました）